Мышковец С.А., v.1 1.05.2023

**РЕШЕНИЕ ЗАДАЧИ**:

Сравнить существующие решения по саммаризации тестов.

**ВЫВОД**:

При оценке саммаризации  сравниваются предикты с кратким содержанием, созданным человеком. Не совсем показательный критерий для абстрактных кратких содержаний.

**Extractive summarazation models**:

Модель facebook/bart-large-cnn обучена на датасете cnn daily и хорошо справляется с саммаризацией новостей.
Модель philschmid/bart-large-cnn-samsum фактически та же модель facebook/bart-large-cnn дотренированная  на датасете samsum и хорошо справляется с саммаризацией диалогов.

**Abstractive summarazation models**:

Все модели хорошо справляются с саммаризацией информационного текста, с диалогами у большинства моделей проблемы.

Модель mT5 Model from Transformers была обучена на отзывах и хорошо справляется с их саммаризацией. Но несмотря на это, если прогнать через модель большое кол-во отзывов, есть сильные смысловые и эмоциональные отклонения.

Модель bert-extractive-summarizer XLNet based Transformers отличилась низкими показателями, но при саммаризации новостной статьи выдала хоть и очень сжатую, но точную выдержку.

Лучше всех с саммаризацией справился chart-gpt4. Модель отлично справляется при любых условиях по длине краткого содержания, не теряя смысла и эмоциональной окраски.


**Результаты самммаризации для новостной статьи:**

| Summarization type| EXTRACTIVE | EXTRACTIVE | ABSTRACTIVE | ABSTRACTIVE | ABSTRACTIVE  | ABSTRACTIVE | ABSTRACTIVE
| :---        |    :----:   |    :----: |   :----:   |    :----:   |    :----:   |    :----:   | :----:   | 
| **Metrics**| **HF facebook/bart-large-cnn** | **philschmid/bart-large-cnn-samsum** | **Google’s T5 Model from Transformers** | **mT5 Model from Transformers** | **bert-extractive-summarizer GPT2 based Transformers**  |**bert-extractive-summarizer XLNet based Transformers**  | **chart-gpt4** 
| origin lenfth | 258 | 258 |258 | 258 | 258 | 258 | 258 |
| summary lenfth |  48 | 54 |42 | 2 | 54 |72 |91|
| rouge1 |  0.28 | 0.38 | 0.41 | 0.03 | 0.46 |0.08 |0.34|
| rouge2 | 0.05 | 0.13 | 0.17 | 0.0 | 0.27 | 0.0 | 0.14|
| rougeL | 0.19 | 0.22 | 0.32 | 0.03 | 0.37| 0.08| 0.30|
| rougeLsum  | 0.21 | 0.30 | 0.34 | 0.03 | 0.33 | 0.08| 0.30 |

**Все показатели качества самммаризации всех рассмотренных моделей для отзывов равны 0. Отзывы сами по себе короткие и требуют еще более сжатого вывода краткого содержания.**

**Результаты самммаризации для диалога:**

| Summarization type| EXTRACTIVE | EXTRACTIVE | ABSTRACTIVE | ABSTRACTIVE | ABSTRACTIVE  | ABSTRACTIVE | ABSTRACTIVE
| :---        |    :----:   |    :----: |   :----:   |    :----:   |    :----:   |    :----:   | :----:   | 
| **Metrics**| **HF facebook/bart-large-cnn** | **philschmid/bart-large-cnn-samsum** | **Google’s T5 Model from Transformers** | **mT5 Model from Transformers** | **bert-extractive-summarizer GPT2 based Transformers**  |**bert-extractive-summarizer XLNet based Transformers**  | **chart-gpt4** 
| origin lenfth | 107 | 107 | 107 | 107 | 107 | 107 | 107 |
| summary lenfth |  49 | 44 | 32 | 2 | 4 | 23 | 61|
| rouge1 |  0.36 | 0.33 | 0.12 | 0.10 | 0.0 |0.0 |0.28 |
| rouge2 | 0.09 | 0.06 | 0.04 | 0.0 | 0.0 | 0.0 | 0.10 |
| rougeL | 0.24 | 0.2 | 0.12 | 0.10 | 0.0| 0.0 | 0.23 |
| rougeLsum  | 0.24 | 0.2 | 0.12 | 0.10 | 0.0 | 0.0| 0.23 |

---

In [408]:
import pandas as pd

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle


import evaluate

---

# Metric Card for ROUGE

Для оценки качества массаризации используется метрика rouge (or Recall-Oriented Understudy for Gisting Evaluation). Эта метрика представляет собой набор показателей и программный пакет, используемый для оценки программного обеспечения для автоматического суммирования и машинного перевода при обработке естественного языка. Метрики сравнивают автоматически созданное краткое содержание или перевод со справочной информацией или набором ссылок (созданным человеком) кратким содержанием или переводом.

ROUGE нечувствителен к регистру, что означает, что буквы верхнего регистра обрабатываются так же, как буквы нижнего регистра.

Эта метрика является оберткой  в Google Research reimplementation of ROUGE.

**Виды метрики rouge:**

"rouge1": unigram (1-gram) based scoring

"rouge2": bigram (2-gram) based scoring

"rougeL": Longest common subsequence based scoring.

"rougeLSum": splits text using "\n"

https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
# pip install evaluate
# !pip install rouge-score

In [49]:
# Пример использования метрики ROUGE
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=predictions,
                         references=references,
                            tokenizer=lambda x: x.split())
print(results)
# {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


---

# Рассмотрим саммаризацию новостной статьи, отзыва с amazon и несложного диалога.

**Обозначим тексты для саммаризации и определим функции для оценки ее качества.**

In [364]:
ARTICLE_bbc_news = """Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide.
Brienz's fewer than 100 villagers were given just 48 hours to pack what they could and abandon their homes.
Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent.
Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.
The development has raised questions about the safety of some mountain communities, as global warming changes the alpine environment.
Brienz, in the eastern canton of Graubünden, is now empty.
The village has been judged a geological risk for some time and is built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings.
As the minutes ticked towards the deadline to leave, even Brienz's dairy cows were being taken to safety.
The residents, some young, some old, families, farmers and professional couples, had two days to abandon their homes.
They were asked earlier this week to evacuate the village by Friday evening.
Switzerland's Alpine regions are especially sensitive to global warming - as the permafrost high in the mountains begins to thaw, the rock becomes more unstable.
This particular mountain has always been unstable, but recently the rock has been shifting faster and faster.
Days of heavy rain could bring two million cubic metres of loosened rock crashing down the mountainside onto the village, scientists warned.
Now the villagers must wait, in temporary accommodation, for the rock to fall - and hope it misses their homes.
"""

In [365]:
human_produced_summary_bbc_news = """Residents of a tiny Swiss village have all been evacuated  within 48 hours due to the risk of an imminent rockslide. 
The residents must wait and hope their homes will not be destroyed by loosened rock crashing down the mountainside. 
The safety of mountain communities has been judged a geological risk for some time and is considered to be a global warming consequence."""

In [366]:
def eval_article_summary(prediction):
    rouge = evaluate.load('rouge')
    predictions = [prediction]
    references = [human_produced_summary_bbc_news]
    results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split() to use for non-latin languages
                            )
    print(f'origin length:', len(ARTICLE_bbc_news.split(' ')))
    print(f'summary length:',len(prediction.split(' ')))
    print(prediction)
    print(results)

In [367]:
REVIEW= """I feel I have to write to keep others from wasting their money. 
This book seems to have been written by a 7th grader with poor grammatical skills for her age! 
As another reviewer points out, there is a misspelling on the cover, and I believe there is at least one per chapter. 
For example, it was mentioned twice that she had a "lean" on her house. 
I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. 
Please don't waste your money. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!
"""

In [368]:
human_produced_summary_review = "Awful beyond belief!"

In [369]:
def eval_review_summary(prediction):
    rouge = evaluate.load('rouge')
    predictions = [prediction]
    references = [human_produced_summary_review]
    results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split() to use for non-latin languages
                            )
    print(f'origin length:', len(REVIEW.split(' ')))
    print(f'summary length:',len(prediction.split(' ')))
    print(prediction)
    print(results)

In [372]:
DIALOGUE = """Jane: Good morning, Doctor Rudra, how are you doing?
Doctor Rudra: Good morning, Jane. I am doing well. And you?
Jane: I’m great, thank you. This is my friend Leila. She is thinking about joining the hospital but she has a few questions about the administration there. Would you mind telling her about the administration, please?
Doctor Rudra: Hello, Leila! It’s a pleasure to meet you. I’m more than happy to speak with you. Please stop by my chamber tomorrow.
Leila: It’s a pleasure to meet you, Doctor. Thank you so much for helping us.
Doctor Rudra: Don’t mention it. Hopefully, I will be able to help you out in this matter.
"""

In [373]:
human_produced_summary_dialogue = ['Jane asked Doctor Rudra to tell her friend Leila about hospital administration before she joins this hospital.']

In [374]:
def eval_dialogue_summary(prediction):
    rouge = evaluate.load('rouge')
    predictions = [prediction]
    references = [human_produced_summary_dialogue]
    results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split() to use for non-latin languages
                            )
    print(f'origin length:', len(DIALOGUE.split(' ')))
    print(f'summary length:',len(prediction.split(' ')))
    print(prediction)
    print(results)

In [ ]:
conversation_test = '''Waiter: Hello, good evening. Can I start you off with some refreshing drink?
Rana: Yes. I’ll have iced tea, please.
Amal: And I’ll have a chocolate cold coffee.
Waiter: Ok. Should I take your order now, or do you need a few minutes more?
Rana: No no we are ready, you can take the order. I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. And, please also bring a bowl of garlic rice.
Waiter: Sure sir. How do you want the chicken— low spicy, medium, or high on spice?
Rana: Medium spice, please.
Amal: And I’ll just have the beef, with bread and a salad.
'''

---

# **Примеры extractive summarization**

#  Выбраны модели с https://huggingface.co/ фильтром MOST LIKES & MOST DOWNLOADS

# facebook/bart-large-cnn

**Model description**
BART is a transformer encoder-encoder (seq2seq) model with a bidirectional (BERT-like) encoder and an autoregressive (GPT-like) decoder. BART is pre-trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text.

BART is particularly effective when fine-tuned for text generation (e.g. summarization, translation) but also works well for comprehension tasks (e.g. text classification, question answering). This particular checkpoint has been fine-tuned on CNN Daily Mail, a large collection of text-summary pairs.

**Intended uses & limitations**
You can use this model for text summarization.

**Evaluation results**

ROUGE-1 on cnn_dailymail self-reported 42.949

ROUGE-2 on cnn_dailymail self-reported 20.815

ROUGE-L on cnn_dailymail self-reported 30.619

ROUGE-LSUM on cnn_dailymail self-reported 40.038 

loss on cnn_dailymail self-reported 2.529

gen_len on cnn_dailymail self-reported 78.587


Предположительно loss:

**criterion label_smoothed_cross_entropy**

https://github.com/facebookresearch/fairseq/blob/main/examples/bart/README.summarization.md#4-fine-tuning-on-cnn-dm-summarization-task
4) Fine-tuning on CNN-DM summarization task:
Example fine-tuning CNN-DM

TOTAL_NUM_UPDATES=20000  
WARMUP_UPDATES=500      
LR=3e-05
MAX_TOKENS=2048
UPDATE_FREQ=4
BART_PATH=/path/to/bart/model.pt

CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 fairseq-train cnn_dm-bin \
    --restore-file $BART_PATH \
    --max-tokens $MAX_TOKENS \
    --task translation \
    --source-lang source --target-lang target \
    --truncate-source \
    --layernorm-embedding \
    --share-all-embeddings \
    --share-decoder-input-output-embed \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --arch bart_large \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.01 --optimizer adam --adam-betas "(0.9, 0.999)" --adam-eps 1e-08 \
    --clip-norm 0.1 \
    --lr-scheduler polynomial_decay --lr $LR --total-num-update $TOTAL_NUM_UPDATES --warmup-updates $WARMUP_UPDATES \
    --fp16 --update-freq $UPDATE_FREQ \
    --skip-invalid-size-inputs-valid-test \
    --find-unused-parameters;

In [ ]:
# pip install transformers

In [210]:
from transformers import pipeline

summarizer_facebook = pipeline("summarization", model="facebook/bart-large-cnn")

**2. Оценим качество моделей.**

In [355]:
for i in summarizer_facebook(ARTICLE_bbc_news, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False):
    for k, v in i.items():
        print(summarizer_facebook(ARTICLE_bbc_news, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False))

[{'summary_text': 'Brienz, in the eastern canton of Graubünden, is now empty. The village is built on land that is subsiding down towards the valley. Two million cubic metres of rock is coming loose from the mountain above. The development has raised questions about the safety of some mountain communities.'}]


In [358]:
eval_article_summary("Brienz, in the eastern canton of Graubünden, is now empty. The village is built on land that is subsiding down towards the valley. Two million cubic metres of rock is coming loose from the mountain above. The development has raised questions about the safety of some mountain communities.")

origin length: 258
summary length: 48
Brienz, in the eastern canton of Graubünden, is now empty. The village is built on land that is subsiding down towards the valley. Two million cubic metres of rock is coming loose from the mountain above. The development has raised questions about the safety of some mountain communities.
{'rouge1': 0.2857142857142857, 'rouge2': 0.05454545454545454, 'rougeL': 0.19642857142857142, 'rougeLsum': 0.2142857142857143}


In [ ]:
# Load a pretrained model - ValueError: No model config found in the file at .
# Saved weights, no model architecture

# import tensorflow as tf
# model = tf.keras.models.load_model('tf_model.h5')

In [370]:
for i in summarizer_facebook(REVIEW, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False):
    for k, v in i.items():
        print(summarizer_facebook(REVIEW, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False))

[{'summary_text': "I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!"}]


In [371]:
eval_review_summary("I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!")

origin length: 129
summary length: 57
I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [375]:
for i in summarizer_facebook(DIALOGUE, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False):
    for k, v in i.items():
        print(summarizer_facebook(DIALOGUE, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False))

[{'summary_text': 'Doctor Rudra is asked to help a friend who is thinking about joining the hospital. Jane asks him to tell her about the administration there. Doctor Rudra says he will be happy to help her out in this matter. He also asks her to stop by his chamber tomorrow.'}]


In [376]:
eval_dialogue_summary('Doctor Rudra is asked to help a friend who is thinking about joining the hospital. Jane asks him to tell her about the administration there. Doctor Rudra says he will be happy to help her out in this matter. He also asks her to stop by his chamber tomorrow.')

origin length: 107
summary length: 49
Doctor Rudra is asked to help a friend who is thinking about joining the hospital. Jane asks him to tell her about the administration there. Doctor Rudra says he will be happy to help her out in this matter. He also asks her to stop by his chamber tomorrow.
{'rouge1': 0.36363636363636365, 'rouge2': 0.09375, 'rougeL': 0.24242424242424243, 'rougeLsum': 0.24242424242424243}


**Перепроверим результат на другом диалоге.**

In [377]:
for i in summarizer_facebook(conversation_test, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False):
    for k, v in i.items():
        print(summarizer_facebook(conversation_test, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False))

[{'summary_text': "Rana: I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. Amal: And I'll just have the beef, with bread and a salad. Rana: How do you want the chicken— low spicy, medium, or high on spice?"}]


In [379]:
eval_dialogue_summary("""Rana: I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. Amal: And I'll just have the beef, with bread and a salad. Rana: How do you want the chicken— low spicy, medium, or high on spice?""")

origin length: 107
summary length: 44
Rana: I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. Amal: And I'll just have the beef, with bread and a salad. Rana: How do you want the chicken— low spicy, medium, or high on spice?
{'rouge1': 0.031746031746031744, 'rouge2': 0.0, 'rougeL': 0.031746031746031744, 'rougeLsum': 0.031746031746031744}


Краткое содержание не имеет смысла.

---

# philschmid/bart-large-cnn-samsum

**Evaluation results**

eval_rouge1	42.621

eval_rouge2	21.9825

eval_rougeL	33.034

eval_rougeLsum	39.6783

test_rouge1	41.3174

test_rouge2	20.8716

test_rougeL	32.1337

test_rougeLsum	38.4149

In [240]:
from transformers import pipeline
summarizer_philschmid= pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [380]:
for i in summarizer_philschmid(ARTICLE_bbc_news):
    for k, v in i.items():
        print(summarizer_philschmid(ARTICLE_bbc_news))

[{'summary_text': 'The residents of Brienz, in the eastern canton of Graubünden, Switzerland, were asked to leave their homes by Friday evening due to the risk of a rockslide. Two million cubic metres of loose rock is coming loose from the mountain and could obliterate the village. Even the dairy cows were loaded up for departure.'}]


In [381]:
eval_article_summary('The residents of Brienz, in the eastern canton of Graubünden, Switzerland, were asked to leave their homes by Friday evening due to the risk of a rockslide. Two million cubic metres of loose rock is coming loose from the mountain and could obliterate the village. Even the dairy cows were loaded up for departure.')

origin length: 258
summary length: 54
The residents of Brienz, in the eastern canton of Graubünden, Switzerland, were asked to leave their homes by Friday evening due to the risk of a rockslide. Two million cubic metres of loose rock is coming loose from the mountain and could obliterate the village. Even the dairy cows were loaded up for departure.
{'rouge1': 0.38983050847457623, 'rouge2': 0.13793103448275862, 'rougeL': 0.22033898305084743, 'rougeLsum': 0.30508474576271183}


In [382]:
for i in summarizer_philschmid(REVIEW):
    for k, v in i.items():
        print(summarizer_philschmid(REVIEW))

[{'summary_text': 'The book seems to have been written by a 7th grader with poor grammatical skills for her age. There is a misspelling on the cover and there are at least one per chapter. I was so distracted by the poor writing and weak plot that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling.'}]


In [383]:
eval_review_summary('The book seems to have been written by a 7th grader with poor grammatical skills for her age. There is a misspelling on the cover and there are at least one per chapter. I was so distracted by the poor writing and weak plot that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling.')

origin length: 129
summary length: 63
The book seems to have been written by a 7th grader with poor grammatical skills for her age. There is a misspelling on the cover and there are at least one per chapter. I was so distracted by the poor writing and weak plot that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling.
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [384]:
for i in summarizer_philschmid(DIALOGUE):
    for k, v in i.items():
        print(summarizer_philschmid(DIALOGUE))

[{'summary_text': "Jane's friend Leila is thinking about joining the hospital. Doctor Rudra will help her with some questions about the administration. Leila will come to his chamber tomorrow to talk to him about it. He will be more than happy to help her.  "}]


In [385]:
eval_dialogue_summary("Jane's friend Leila is thinking about joining the hospital. Doctor Rudra will help her with some questions about the administration. Leila will come to his chamber tomorrow to talk to him about it. He will be more than happy to help her.  ")

origin length: 107
summary length: 44
Jane's friend Leila is thinking about joining the hospital. Doctor Rudra will help her with some questions about the administration. Leila will come to his chamber tomorrow to talk to him about it. He will be more than happy to help her.  
{'rouge1': 0.3333333333333333, 'rouge2': 0.06896551724137931, 'rougeL': 0.2, 'rougeLsum': 0.2}


**Перепроверим результат на другом диалоге**

In [386]:
for i in summarizer_philschmid(conversation_test):
    for k, v in i.items():
        print(summarizer_philschmid(conversation_test))

[{'summary_text': "Rana will have the corn mushroom soup, grilled chicken with mashed potatoes and peas and garlic rice. Amal will have beef, with bread and salad, and a chocolate cold coffee. Waiter will take Rana's order and bring the order to Amal."}]


In [387]:
eval_dialogue_summary("Rana will have the corn mushroom soup, grilled chicken with mashed potatoes and peas and garlic rice. Amal will have beef, with bread and salad, and a chocolate cold coffee. Waiter will take Rana's order and bring the order to Amal.")

origin length: 107
summary length: 41
Rana will have the corn mushroom soup, grilled chicken with mashed potatoes and peas and garlic rice. Amal will have beef, with bread and salad, and a chocolate cold coffee. Waiter will take Rana's order and bring the order to Amal.
{'rouge1': 0.033898305084745756, 'rouge2': 0.0, 'rougeL': 0.033898305084745756, 'rougeLsum': 0.033898305084745756}


---

---

# **Abstract summarization**

# Google’s T5 Model from Transformers

In [256]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [257]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# pip install sentencepiece

In [258]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [259]:
inputs = tokenizer.encode(ARTICLE_bbc_news, return_tensors='pt', max_length=512, truncation=True) 

In the above code we have used encode() method and passed following parameters -

return_tensors: If set, will return tensors instead of a list of python integers. We can also use “tf” for TensorFlow, “pt” for PyTorch and “np” for Numpy.

max_length: Controls the maximum length to use by one of the truncation/padding parameters.

truncation: Activates and controls truncation.

The max_length and truncation parameters together indicate that we do not want the original text to bypass 512 tokens, which is the default limit set for tokenization in Transformers.

In [260]:
output = model.generate(inputs, 
                        min_length=40, 
                        max_length=200, 
                        length_penalty=1, 
                        num_beams=4, 
                        early_stopping=True
                        )

In [261]:
sequences = tokenizer.batch_decode(output)
sequences

['<pad> residents of Brienz, in the eastern canton of Graubünden, have all been evacuated because of the risk of an imminent rockslide. two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.</s>']

In [388]:
eval_article_summary('residents of Brienz, in the eastern canton of Graubünden, have all been evacuated because of the risk of an imminent rockslide. two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.')

origin length: 258
summary length: 41
residents of Brienz, in the eastern canton of Graubünden, have all been evacuated because of the risk of an imminent rockslide. two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.
{'rouge1': 0.41904761904761906, 'rouge2': 0.17475728155339806, 'rougeL': 0.32380952380952377, 'rougeLsum': 0.34285714285714286}


In [271]:
inputs = tokenizer.encode(REVIEW, return_tensors='pt', max_length=512, truncation=True) 

In [272]:
output = model.generate(inputs, 
                        min_length=10, 
                        max_length=30, 
                        length_penalty=1, 
                        num_beams=4, 
                        early_stopping=True
                        )

In [273]:
sequences = tokenizer.batch_decode(output)
sequences

['<pad><extra_id_0>I was so distracted by the poor writing and weak plot, that I decided to read with a pencil to mark all of the horrible grammar']

In [389]:
eval_review_summary('I was so distracted by the poor writing and weak plot, that I decided to read with a pencil to mark all of the horrible grammar')

origin length: 129
summary length: 26
I was so distracted by the poor writing and weak plot, that I decided to read with a pencil to mark all of the horrible grammar
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [277]:
inputs = tokenizer.encode(DIALOGUE, return_tensors='pt', max_length=512, truncation=True) 

In [278]:
output = model.generate(inputs, 
                        min_length=10, 
                        max_length=50, 
                        length_penalty=1, 
                        num_beams=4, 
                        early_stopping=True
                        )

In [279]:
sequences = tokenizer.batch_decode(output)
sequences

['<pad><extra_id_0>I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning']

In [390]:
eval_dialogue_summary("""I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning""")

origin length: 107
summary length: 32
I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning, Jane. I am doing well. And you? Doctor Rudra: Good morning
{'rouge1': 0.12244897959183672, 'rouge2': 0.0425531914893617, 'rougeL': 0.12244897959183672, 'rougeLsum': 0.12244897959183672}


Краткое содержание диалога смысла не имеет.

**Перепроверим результат на другом диалоге**

In [283]:
inputs = tokenizer.encode(conversation_test, return_tensors='pt', max_length=512, truncation=True) 
output = model.generate(inputs, 
                        min_length=10, 
                        max_length=50, 
                        length_penalty=1, 
                        num_beams=4, 
                        early_stopping=True
                        )
sequences = tokenizer.batch_decode(output)
sequences

['<pad><extra_id_0>Can I start you off with some refreshing drink? Rana: Yes. Amal: And I’ll have a chocolate cold coffee. Waiter: Hello, good evening. Can I take the order now? Rana: Yes']

По-прежнему, качество саммаризации оставляет желать лучшего. 

---

# HF mT5 Model from Transformers

In [335]:
from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=hub_model_id, max_length=60)

In [336]:
summarizer(ARTICLE_bbc_news)

[{'summary_text': 'Accidental rockslide'}]

In [391]:
eval_article_summary("Accidental rockslide")

origin length: 258
summary length: 2
Accidental rockslide
{'rouge1': 0.03076923076923077, 'rouge2': 0.0, 'rougeL': 0.03076923076923077, 'rougeLsum': 0.03076923076923077}


In [341]:
summarizer(REVIEW)

[{'summary_text': 'Poor grammar and spelling'}]

In [392]:
eval_review_summary('Poor grammar and spelling')

origin length: 129
summary length: 4
Poor grammar and spelling
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


Модель показывает достаточно интересные результаты.
Проверим на большой выборке.

In [411]:
df = pd.read_csv('model_predicted_summary_train.csv')

In [412]:
df = shuffle(df)

In [416]:
df

,score,summary,review,model predicted summary
15735,4.0,Fast and in good shape,It Came to me pretty fast and was in good shap...,Very happy with it
12750,3.0,It ain't Studs Terkel,Noble try. Right instincts. There must surely ...,Very interesting
19702,4.0,Like the info but...,"really, really, REALLY hate the difficult to r...","Really, really, REALLY hate the difficult to r..."
24786,5.0,"Excellent book, a sigh of relief!",I'd just like to say that i am extreamly pleas...,Very helpful
11225,3.0,1984 has come and gone...,Orwell's &quot;1984&quot; is a classic piece o...,Very heavy-handed approach to storytelling
...,...,...,...,...
18630,4.0,Fahrenheit 451,I admit I have never read this important book....,Very interesting
4522,1.0,"Forbush a false, carnal teacher about Jesus' life",Forbush equates good Judaism and Christianity ...,"Very interesting, but a very interesting read."
2014,1.0,Erotica for sci-fi geeks,This is a collection of some computer geek's w...,Very funny
12109,3.0,Enjoyable if you know the history behind this ...,I did not enjoy this story. I decided to read ...,I did not enjoy this story. I did not like any...


Можно видеть, что некоторые результаты достаточно противоречивы (краткое содержание id 4522, 2014, 6632 явно не совпадает с кратким содержание автора и противоречит поставленной оценке).

In [346]:
summarizer(DIALOGUE)

[{'summary_text': 'Good, Jane'}]

In [393]:
eval_dialogue_summary('Good, Jane')

origin length: 107
summary length: 2
Good, Jane
{'rouge1': 0.10526315789473684, 'rouge2': 0.0, 'rougeL': 0.10526315789473684, 'rougeLsum': 0.10526315789473684}


Краткое содержание диалога смысла не имеет.

---

# bert-extractive-summarizer GPT2 based Transformers

In [ ]:
# pip install bert-extractive-summarizer

In [287]:
from summarizer import Summarizer,TransformerSummarizer

In [288]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="distilgpt2")

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [296]:
full = ''.join(GPT2_model(ARTICLE_bbc_news, min_length=60))
print(full)

Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide. Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent. Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.


/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [394]:
eval_article_summary('Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide. Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent. Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.')

origin length: 258
summary length: 54
Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide. Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent. Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.
{'rouge1': 0.4615384615384615, 'rouge2': 0.24347826086956523, 'rougeL': 0.37606837606837606, 'rougeLsum': 0.37606837606837606}


In [299]:
full = ''.join(GPT2_model(REVIEW, 
                          # min_length=5, 
                          max_length=60
                        ))
print(full)

I will not put much faith in the reviews from now on!


In [395]:
eval_review_summary('I will not put much faith in the reviews from now on!')

origin length: 129
summary length: 12
I will not put much faith in the reviews from now on!
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [304]:
full = ''.join(GPT2_model(DIALOGUE, min_length=9, max_length=20))
print(full)

I am doing well.


In [396]:
eval_dialogue_summary('I am doing well.')

origin length: 107
summary length: 4
I am doing well.
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


Краткое содержание диалога смысла не имеет.

**Перепроверим результат на другом диалоге**

In [307]:
full = ''.join(GPT2_model(conversation_test, min_length=9, max_length=20))
print(full)

Rana: Yes. Waiter: Ok.


/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Краткое содержание диалога по-прежему смысла не имеет.

---

# bert-extractive-summarizer XLNet based Transformers

In [308]:
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-large-cased")

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [320]:
full = ''.join(model(ARTICLE_bbc_news, min_length=40, max_length=70))
print(full)

Brienz, in the eastern canton of Graubünden, is now empty.


In [397]:
eval_article_summary('Brienz, in the eastern canton of Graubünden, is now empty.')

origin length: 258
summary length: 10
Brienz, in the eastern canton of Graubünden, is now empty.
{'rouge1': 0.08108108108108109, 'rouge2': 0.0, 'rougeL': 0.08108108108108109, 'rougeLsum': 0.08108108108108109}


In [321]:
full = ''.join(model(REVIEW, 
                     min_length=3,
                    #  max_length=5
                     ))
print(full)

I feel I have to write to keep others from wasting their money. This book seems to have been written by a 7th grader with poor grammatical skills for her age!


/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [398]:
eval_review_summary('I feel I have to write to keep others from wasting their money. This book seems to have been written by a 7th grader with poor grammatical skills for her age!')

origin length: 129
summary length: 31
I feel I have to write to keep others from wasting their money. This book seems to have been written by a 7th grader with poor grammatical skills for her age!
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [325]:
full = ''.join(model(DIALOGUE, min_length=10))
print(full)

Jane: Good morning, Doctor Rudra, how are you doing? It’s a pleasure to meet you. Leila: It’s a pleasure to meet you, Doctor.


/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [399]:
eval_dialogue_summary('Jane: Good morning, Doctor Rudra, how are you doing? It’s a pleasure to meet you. Leila: It’s a pleasure to meet you, Doctor.')

origin length: 107
summary length: 23
Jane: Good morning, Doctor Rudra, how are you doing? It’s a pleasure to meet you. Leila: It’s a pleasure to meet you, Doctor.
{'rouge1': 0.2380952380952381, 'rouge2': 0.05, 'rougeL': 0.2380952380952381, 'rougeLsum': 0.2380952380952381}


Краткое содержание диалога смысла не имеет.

**Перепроверим результат на другом диалоге**

In [327]:
full = ''.join(model(conversation_test, min_length=10))
print(full)

Waiter: Hello, good evening. Amal: And I’ll have a chocolate cold coffee. Rana: No no we are ready, you can take the order.


/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Краткое содержание диалога по-прежему смысла не имеет.

---

# chart-gpt4

На сайте https://chat-gpt.ru/ попросила предоставить резюме по текстам, обозначенным в начале блокнота. Длину краткого содержания не ограничивала. Ниже результаты:

**по новостной статье:**

Residents of the Swiss village Brienz have been evacuated due to risk of an imminent rock slide. The village was built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings. Scientists have warned that two million cubic metres of loosened rock could fall down the mountainside onto the village due to heavy rain. Switzerland's Alpine regions are particularly sensitive to global warming, as the permafrost high in the mountains begins to thaw, causing the rock to become unstable.

In [292]:
chart_gpt4_predict = """Residents of the Swiss village Brienz have been evacuated due to risk of an imminent rock slide. The village was built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings. Scientists have warned that two million cubic metres of loosened rock could fall down the mountainside onto the village due to heavy rain. Switzerland's Alpine regions are particularly sensitive to global warming, as the permafrost high in the mountains begins to thaw, causing the rock to become unstable."""

In [400]:
eval_article_summary(chart_gpt4_predict)

origin length: 258
summary length: 91
Residents of the Swiss village Brienz have been evacuated due to risk of an imminent rock slide. The village was built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings. Scientists have warned that two million cubic metres of loosened rock could fall down the mountainside onto the village due to heavy rain. Switzerland's Alpine regions are particularly sensitive to global warming, as the permafrost high in the mountains begins to thaw, causing the rock to become unstable.
{'rouge1': 0.3483870967741936, 'rouge2': 0.14379084967320263, 'rougeL': 0.30967741935483867, 'rougeLsum': 0.30967741935483867}


Результат с ограничением по длине до 50 слов.

Residents of a small Swiss village have been evacuated due to the risk of a rockslide, giving them just 48 hours to leave their homes. Brienz, located in the Graubünden region, is at risk due to subsiding land causing cracks. The village is now empty, with even dairy cows being taken to safety. Global warming is also said to be contributing to the risk in Alpine regions, with melting permafrost making rocks more unstable. Two million cubic metres of rock is at risk of falling onto the village, with residents waiting in temporary accommodation.

**по отзыву:**

Book poorly written, don't buy.

In [402]:
eval_review_summary("Book poorly written, don't buy.")

origin length: 129
summary length: 5
Book poorly written, don't buy.
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


Ниже результат с ограничением по длине до 5 слов.

Book poorly written with weak plot.

**по диалогу:**

Jane greets Doctor Rudra and introduces her friend Leila who is interested in joining the hospital. Jane asks Doctor Rudra if he can provide information about the hospital administration. Doctor Rudra agrees and invites Leila to meet him in his chamber the following day. Leila thanks Doctor Rudra for his help and Doctor Rudra hopes to provide helpful information to her.

In [403]:
eval_dialogue_summary('Jane greets Doctor Rudra and introduces her friend Leila who is interested in joining the hospital. Jane asks Doctor Rudra if he can provide information about the hospital administration. Doctor Rudra agrees and invites Leila to meet him in his chamber the following day. Leila thanks Doctor Rudra for his help and Doctor Rudra hopes to provide helpful information to her.')

origin length: 107
summary length: 61
Jane greets Doctor Rudra and introduces her friend Leila who is interested in joining the hospital. Jane asks Doctor Rudra if he can provide information about the hospital administration. Doctor Rudra agrees and invites Leila to meet him in his chamber the following day. Leila thanks Doctor Rudra for his help and Doctor Rudra hopes to provide helpful information to her.
{'rouge1': 0.2820512820512821, 'rouge2': 0.10526315789473685, 'rougeL': 0.23076923076923075, 'rougeLsum': 0.23076923076923075}


Two customers order drinks and dinner at a restaurant, with one requesting iced tea and corn mushroom soup, and the other requesting a chocolate cold coffee and beef.

Ниже результат по запросу с ограничением по длине до 40 слов.

Jane greets Doctor Rudra and introduces her friend Leila, who is interested in joining the hospital. Doctor Rudra offers to speak with Leila about the administration and invites her to visit his chamber the next day. Leila thanks him and hopes he can help her.